In [1]:
'''
Run on virtual environment: Snow_env with the following packages installed
***tested on Python 3.9.12, does not work with python 3.10***

Load all dependencies via cmd and pay special attention to version requirements.

Steps for installing virtual env: https://janakiev.com/blog/jupyter-virtual-envs/

1st geopandas: https://towardsdatascience.com/geopandas-installation-the-easy-way-for-windows-31a666b3610f
Here are binaries: https://www.lfd.uci.edu/~gohlke/pythonlibs/
2nd: rioxarray
3rd: rasterio
tables
sklearn
earthpy
numpy 1.22
pyshp 2.1
h5py
tqdm
matplotlib 3.5
seaborn
tensorflow



joblib
pillow
panda
pyproj
shapely
kiwisolver
fonttools
pillow
mpl_toolkits - pip install --user basemap
**git clone https://github.com/matplotlib/matplotlib
cd matplotlib
pip install -e .**
tables
pytables
netCDF4 **https://unidata.github.io/netcdf4-python/

Data Assimilation of snotel data
https://snowex-hackweek.github.io/website/tutorials/geospatial/SNOTEL_query.html

'''

import os
import National_Snow_Model
import NSM_SCA
import pandas as pd
import warnings
from datetime import date, timedelta
warnings.filterwarnings("ignore")

C:\Users\rjohnson18\Anaconda3\envs\NSM_env\lib\site-packages\geopandas\_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(
C:\Users\rjohnson18\Anaconda3\envs\NSM_env\lib\site-packages\xarray\backends\cfgrib_.py:29: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


In [2]:
#set path directory
os.getcwd()
os.chdir('..')  # TODO find a way around this, it causes issues when rerunning the code
cwd = os.getcwd()
cwd

'C:\\Users\\rjohnson18\\Box\\CIROH\\SnowModeling\\Snow_Modeling_Group\\NSM_App_Deployment\\National-Snow-Model'

## Running the NSM to create retrospective datasets



In [12]:
#Grab existing files based on water year
prev_year = '2022'
prev_date = prev_year + '-09-24'

#input the new water year of choice
new_year = '2018'
new_date = new_year + '-09-24'

#threshold
threshold = '20.0'

#write code for CSV files

#for h5 files
Region_list = ['N_Sierras',
                       'S_Sierras_High',
                       'S_Sierras_Low',
                       'Greater_Yellowstone',
                       'N_Co_Rockies',
                       'SW_Mont',
                       'SW_Co_Rockies',
                       'GBasin',
                       'N_Wasatch',
                       'N_Cascade',
                       'S_Wasatch',
                       'SW_Mtns',
                       'E_WA_N_Id_W_Mont',
                       'S_Wyoming',
                       'SE_Co_Rockies',
                       'Sawtooth',
                       'Ca_Coast',
                       'E_Or',
                       'N_Yellowstone',
                       'S_Cascade',
                       'Wa_Coast',
                       'Greater_Glacier',
                       'Or_Coast'
                      ]
SWE_new = {}
for region in Region_list:
    #The below file will serve as a starting poinw
    SWE_new[region] = pd.read_hdf(cwd+f"/Predictions/predictions{prev_year}-09-24.h5", key = region)
    SWE_new[region].rename(columns = {prev_date:new_date}, inplace = True)
    SWE_new[region].to_hdf(cwd+f"/Predictions/{threshold}/predictions{new_year}-09-24.h5", key = region)
print('.h5 file complete')
    
#set the ground measures features DF    
obs_old = pd.read_csv(cwd+f"/Data/Pre_Processed_DA/ground_measures_features_{prev_year}-09-24.csv")
obs_old.rename(columns = {'Unnamed: 0':'station_id', prev_date:new_date}, inplace = True)
obs_old.set_index('station_id', inplace = True)
obs_old[new_date] = 0
obs_old.to_csv(cwd+f"/Data/Pre_Processed_DA/ground_measures_features_{new_year}-09-24.csv")

print('Ground measures features df complete')

#load the ground_measures_features meta w/preds
obs_meta_old = pd.read_csv(cwd+f"/Data/Processed/DA_ground_measures_features_{prev_year}-09-24.csv")
obs_meta_old.rename(columns = {'Unnamed: 0':'station_id'}, inplace = True)
obs_meta_old.set_index('station_id', inplace = True)
obs_meta_old['Date'] = new_date
obs_meta_old.to_csv(cwd+f"/Data/Processed/DA_ground_measures_features_{new_year}-09-24.csv")

print('Ground measures features meta df complete')

#Make a submission DF
old_preds = pd.read_csv(cwd+ f"/Predictions/submission_format_{prev_date}.csv")
old_preds['2022-10-01'] = 0
new_preds_date = new_year+'-10-01'
old_preds.rename(columns = {'2022-10-01':new_preds_date}, inplace = True)
old_preds.set_index('cell_id', inplace = True)
old_preds.to_csv(cwd+ f"/Predictions/{threshold}/submission_format_{new_date}.csv")
print('New simulation start files complete')

.h5 file complete
Ground measures features df complete
Ground measures features meta df complete
New simulation start files complete


In [5]:
#can be altered to create list every n number of days by changing 7 to desired skip length
def daterange(start_date, end_date):
     for n in range(0, int((end_date - start_date).days) + 1, 7):
        yield start_date + timedelta(n)
         
#define start and end date for list of dates
start_dt = date(2018, 10, 1)
end_dt = date(2019, 7, 31)

#create empty list to store dates
datelist = []

#append dates to list
for dt in daterange(start_dt, end_dt):
    #print(dt.strftime("%Y-%m-%d"))
    dt=dt.strftime('%Y-%m-%d')
    datelist.append(dt)
    
datelist

['2018-10-01',
 '2018-10-08',
 '2018-10-15',
 '2018-10-22',
 '2018-10-29',
 '2018-11-05',
 '2018-11-12',
 '2018-11-19',
 '2018-11-26',
 '2018-12-03',
 '2018-12-10',
 '2018-12-17',
 '2018-12-24',
 '2018-12-31',
 '2019-01-07',
 '2019-01-14',
 '2019-01-21',
 '2019-01-28',
 '2019-02-04',
 '2019-02-11',
 '2019-02-18',
 '2019-02-25',
 '2019-03-04',
 '2019-03-11',
 '2019-03-18',
 '2019-03-25',
 '2019-04-01',
 '2019-04-08',
 '2019-04-15',
 '2019-04-22',
 '2019-04-29',
 '2019-05-06',
 '2019-05-13',
 '2019-05-20',
 '2019-05-27',
 '2019-06-03',
 '2019-06-10',
 '2019-06-17',
 '2019-06-24',
 '2019-07-01',
 '2019-07-08',
 '2019-07-15',
 '2019-07-22',
 '2019-07-29']

## Run model

In [ ]:
#run the model through all time (data acqusition already completed)
for date in datelist:
    print('Updating SWE predictions for ', date)
    #connect interactive script to Wasatch Snow module
    Snow = NSM_SCA.NSM_SCA(cwd, date, threshold=0.2)
    
    #Go get SNOTEL observations -- currently saving to csv, change to H5,
    #dd if self.data < 11-1-2022 and SWE = -9999, 
    #Snow.Get_Monitoring_Data_Threaded()

    #Get the prediction extent
    bbox = Snow.getPredictionExtent()

    #Initialize/Download the granules
    Snow.initializeGranules(bbox, Snow.SCA_folder)

    #Process observations into Model prediction ready format,
    Snow.Data_Processing()

    #Agument with SCA
    Snow.augmentPredictionDFs()

    #Make predictions
    Snow.SWE_Predict()

    #Make CONUS netCDF file, compressed.
    #Snow.netCDF_compressed(plot=False)

    #Make GeoDataframe and plot, self.Geo_df() makes the geo df
    # Snow.Geo_df()
    # Snow.plot_interactive_SWE_comp(pinlat = 39.1, pinlong = -120, web = True)

Updating SWE predictions for  2018-10-01


100%|██████████| 1/1 [00:00<?, ?it/s]


Regional data QA/QC


100%|██████████| 22/22 [00:17<00:00,  1.24it/s]


Calculating mean SCA for each geometry in each region...


...

Calculating mean SCA for each geometry in each region...


100%|██████████| 23/23 [00:52<00:00,  2.29s/it]


Making predictions for:  2018-10-08
N_Sierras
C:\Users\rjohnson18\Box\CIROH\SnowModeling\Snow_Modeling_Group\NSM_App_Deployment\National-Snow-Model/Model/Prev_SWE_Models_Final/N_Sierras/N_Sierras_model.h5
60/60 [==============================] - 0s 3ms/step
S_Sierras_High
C:\Users\rjohnson18\Box\CIROH\SnowModeling\Snow_Modeling_Group\NSM_App_Deployment\National-Snow-Model/Model/Prev_SWE_Models_Final/S_Sierras_High/S_Sierras_High_model.h5
7/7 [==============================] - 0s 2ms/step
S_Sierras_Low
C:\Users\rjohnson18\Box\CIROH\SnowModeling\Snow_Modeling_Group\NSM_App_Deployment\National-Snow-Model/Model/Prev_SWE_Models_Final/S_Sierras_Low/S_Sierras_Low_model.h5
3/3 [==============================] - 0s 2ms/step
Greater_Yellowstone
C:\Users\rjohnson18\Box\CIROH\SnowModeling\Snow_Modeling_Group\NSM_App_Deployment\National-Snow-Model/Model/Prev_SWE_Models_Final/Greater_Yellowstone/Greater_Yellowstone_model.h5
76/76 [==============================] - 0s 2ms/step
N_Co_Rockies
C:\Users\r

5 out of the last 34 calls to <function Model.make_predict_function.<locals>.predict_function at 0x0000027658E461F0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


1/1 [==============================] - 0s 64ms/step


In [8]:
Snow.threshold

20.0

In [ ]:
def run(cwd, date):
    print('Updating SWE predictions for ', date)
    #connect interactive script to Wasatch Snow module
    Snow = NSM_SCA.NSM_SCA(cwd, date)

    #Go get SNOTEL observations -- currently saving to csv, change to H5,
    #dd if self.data < 11-1-2022 and SWE = -9999,
    Snow.Get_Monitoring_Data_Threaded()

    #Get the prediction extent
    bbox = Snow.getPredictionExtent()

    #Initialize/Download the granules
    Snow.initializeGranules(bbox, Snow.SCA_folder)

    #Process observations into Model prediction ready format,
    Snow.Data_Processing()

    #Agument with SCA
    Snow.augmentPredictionDFs()

    #Make predictions
    Snow.SWE_Predict()

    #Make CONUS netCDF file, compressed.
    Snow.netCDF_compressed(plot=False)

In [ ]:
for date in datelist[:4]:
    %time run(cwd, date)